<a href="https://colab.research.google.com/github/Pasupuletibhanupriya/Fmml-labs/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
Answer1. The accuracy of the validation set can be influenced by changing the percentage of the data allocated to it. Here's how it typically works:

### Increasing the Percentage of the Validation Set:
- **More Data for Validation**: When you allocate a higher percentage of the data to the validation set, the model's performance evaluation can become more reliable because it is tested on a more comprehensive sample. This can lead to a more accurate estimate of how well the model generalizes to unseen data.
- **Less Data for Training**: However, increasing the validation set size reduces the amount of data available for training. With less training data, the model may not learn as effectively, potentially leading to poorer performance. If the model becomes undertrained, the validation accuracy could decrease.
  
### Reducing the Percentage of the Validation Set:
- **Less Data for Validation**: A smaller validation set might lead to an unstable or less reliable estimation of the model's accuracy. The smaller the validation set, the more likely it is that random variations in the data could affect the accuracy score, leading to over- or underestimation of the model's true performance.
- **More Data for Training**: On the other hand, reducing the validation set size increases the data available for training. More training data usually leads to better learning, which can improve the model’s overall performance. However, if the validation set is too small, the accuracy might not reflect the model’s true generalization ability.

### Key Considerations:
- **Balance**: There is a trade-off between the amount of data used for training and validation. Ideally, you want enough data in both sets to ensure that the model trains effectively and that the validation accuracy is a reliable indicator of performance.
- **Model Complexity**: The impact of changing the validation set size might be more pronounced for complex models, which require more data to learn effectively.

In practice, it is common to use around 10-20% of the dataset for validation, but the ideal split can vary depending on the specific problem, dataset size, and model complexity.

2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
Answer2:-The size of the training and validation sets plays a crucial role in predicting the accuracy and generalization ability of a machine learning model. Here's how each affects the predictability of accuracy:

### 1. **Training Set Size**:
   - **Larger Training Set**:
     - **Better Model Learning**: A larger training set provides the model with more examples to learn from, which generally leads to better learning and improved performance. The model can capture more patterns, reduce overfitting, and generalize better to new data.
     - **Improved Accuracy**: With more data to learn from, the model is likely to perform better not only on the training set but also on unseen data, leading to a higher and more reliable prediction of accuracy.
   - **Smaller Training Set**:
     - **Limited Learning**: A smaller training set may not provide enough examples for the model to learn effectively, which can lead to underfitting. The model might not capture the underlying patterns of the data, leading to lower accuracy.
     - **Inaccurate Prediction of Generalization**: With limited data, the model might not generalize well to new data, and the accuracy predicted during training might not reflect real-world performance.

### 2. **Validation Set Size**:
   - **Larger Validation Set**:
     - **More Reliable Accuracy Estimate**: A larger validation set means the model's performance is evaluated on a more representative and varied sample of the data, leading to a more reliable estimate of accuracy. It reduces the variance in the accuracy metric, providing a better prediction of how well the model will perform on unseen data.
     - **Possible Trade-off with Training Data**: However, if increasing the validation set size significantly reduces the training set size, it could lead to poorer model learning and affect accuracy.
   - **Smaller Validation Set**:
     - **Higher Variance in Accuracy Estimate**: A smaller validation set may lead to an inaccurate or less reliable estimate of the model's accuracy. The smaller sample might not represent the data distribution well, causing the accuracy to fluctuate more due to random noise or outliers.
     - **Potential Overestimation**: A smaller validation set might result in an overly optimistic accuracy estimate, especially if it does not capture the full variability of the data. This can lead to overconfidence in the model's performance when applied to new, unseen data.

### Key Takeaways:
- **Balance is Crucial**: There needs to be a balance between the training and validation set sizes. Too small of a training set might limit model learning, while too small of a validation set might lead to an unreliable accuracy estimate.
- **Dataset Size Matters**: For small datasets, it's particularly important to carefully choose the split because the trade-offs are more pronounced. For large datasets, the impact of the split size may be less critical, but it's still important to ensure that the validation set is representative.
- **Cross-Validation**: In cases where dataset size is limited, techniques like cross-validation can help in obtaining a more reliable estimate of accuracy by using different portions of the data for training and validation across multiple iterations. This helps in better predicting the model's generalization performance.
3. A common and effective split is to reserve **20% of your dataset for validation** and use the remaining 80% for training. This split often provides a good balance between having enough data to train the model while also having enough data to validate its performance.

However, the exact percentage can depend on the size and nature of your dataset:

- **Smaller datasets**: You might reserve a smaller percentage (e.g., 10%) for validation because you need more data for training.
- **Larger datasets**: With a large dataset, even a smaller fraction like 10-15% might still provide a substantial validation set.
- **High variance in data**: If your data is very diverse or has high variance, you might reserve more (e.g., 25-30%) to ensure your validation set is representative.

The key is ensuring that the validation set is representative of the data the model will encounter in real-world use, without starving the model of training data.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.
Answer:-To implement and compare the accuracy of a 1-Nearest Neighbor (1-NN) classifier and a 3-Nearest Neighbor (3-NN) classifier using the `KNeighborsClassifier` class from scikit-learn, follow these steps:

### 1. Import Necessary Libraries
```python
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
```

### 2. Load and Split the Dataset
For this example, we'll use the Iris dataset.
```python
# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
```

### 3. Train and Test the 1-Nearest Neighbor Classifier
```python
# Initialize the 1-NN classifier
knn_1 = KNeighborsClassifier(n_neighbors=1)

# Train the classifier
knn_1.fit(X_train, y_train)

# Predict the labels on the test set
y_pred_1 = knn_1.predict(X_test)

# Calculate the accuracy of the 1-NN classifier
accuracy_1 = accuracy_score(y_test, y_pred_1)
print(f"1-NN Classifier Accuracy: {accuracy_1:.2f}")
```

### 4. Train and Test the 3-Nearest Neighbor Classifier
```python
# Initialize the 3-NN classifier
knn_3 = KNeighborsClassifier(n_neighbors=3)

# Train the classifier
knn_3.fit(X_train, y_train)

# Predict the labels on the test set
y_pred_3 = knn_3.predict(X_test)

# Calculate the accuracy of the 3-NN classifier
accuracy_3 = accuracy_score(y_test, y_pred_3)
print(f"3-NN Classifier Accuracy: {accuracy_3:.2f}")
```

### 5. Compare the Results
Now that you've trained both classifiers and calculated their accuracies, you can compare them to see which one performs better on the test dataset.

### Complete Code:
```python
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1-NN Classifier
knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_1.fit(X_train, y_train)
y_pred_1 = knn_1.predict(X_test)
accuracy_1 = accuracy_score(y_test, y_pred_1)
print(f"1-NN Classifier Accuracy: {accuracy_1:.2f}")

# 3-NN Classifier
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_3.fit(X_train, y_train)
y_pred_3 = knn_3.predict(X_test)
accuracy_3 = accuracy_score(y_test, y_pred_3)
print(f"3-NN Classifier Accuracy: {accuracy_3:.2f}")
```

### Explanation:
- **KNeighborsClassifier**: This class implements the k-nearest neighbors algorithm, where `n_neighbors` specifies the number of neighbors to consider.
- **accuracy_score**: This function calculates the accuracy of the classifier by comparing the predicted labels with the true labels.

### Expected Outcome:
Running the code will give you the accuracy of both the 1-NN and 3-NN classifiers, allowing you to compare their performances. Typically, 3-NN might perform better due to considering multiple neighbors, which can reduce the effect of noise in the data. However, this depends on the dataset and the underlying distribution.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Answer:-Yes, averaging validation accuracy across multiple splits, such as in cross-validation, generally provides more consistent and reliable results. Here's why:

1. **Reduces Variability**: Single splits can lead to results that are dependent on the particular division of data, which might not fully represent the entire dataset's distribution. By averaging over multiple splits, you reduce the impact of any one particular split and get a more stable estimate of model performance.

2. **Better Generalization Estimate**: Averaging across multiple splits helps in getting a better estimate of the model's ability to generalize to unseen data. This is because the model is evaluated on different subsets of the data, ensuring that the evaluation isn't biased by any specific subset.

3. **Mitigates Overfitting**: If you rely on a single validation split, there's a higher risk of overfitting to that specific split. Multiple splits ensure that the model's performance is assessed in varied scenarios, helping to identify overfitting more effectively.

In summary, averaging validation accuracy across multiple splits is a common practice to achieve more consistent and generalizable results.
2. Does it give more accurate estimate of test accuracy?
Answer:-It seems like your question is missing some context. Could you clarify what "it" refers to? Are you asking about a specific method, model, or tool for estimating test accuracy? Providing more details will help me give you a more accurate answer.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
Answer:-The effect of the number of iterations on an estimate is closely related to the concept of convergence in iterative methods or the Law of Large Numbers in statistical sampling.

### Iterative Methods
In numerical algorithms, such as those used to solve equations or optimize functions, the number of iterations typically affects the accuracy of the estimate.

- **Few Iterations:** With a small number of iterations, the estimate is likely to be less accurate, as the method hasn't fully converged to the true solution.
- **More Iterations:** As the number of iterations increases, the estimate generally becomes more accurate, assuming the method is converging. However, after a certain point, additional iterations may provide diminishing returns, especially if the method is slow to converge or if it has already reached a sufficient level of precision.

### Statistical Sampling (e.g., Monte Carlo Simulations)
In the context of statistical methods or Monte Carlo simulations, the number of iterations (or samples) also influences the quality of the estimate.

- **Few Iterations/Samples:** With a smaller number of samples, the estimate may be highly variable and less reliable because it’s based on limited information.
- **More Iterations/Samples:** As the number of iterations increases, the estimate tends to become more stable and closer to the true value, as more information is being averaged out. This aligns with the Law of Large Numbers, which states that as the number of trials increases, the sample mean will converge to the expected value.

### Key Takeaway
Yes, typically, a higher number of iterations leads to a better estimate. However, the improvement in accuracy may diminish after a certain point, depending on the problem. In practice, you often need to balance between computational cost and the desired level of accuracy.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Answer:-When dealing with a very small training or validation dataset, simply increasing the number of iterations (or epochs) may not be sufficient to improve the model's performance. Here's why:

### 1. **Overfitting Risk**:
   - With a small dataset, the model is likely to overfit quickly. Overfitting occurs when the model learns the noise and details in the training data rather than the underlying pattern, leading to poor generalization to new, unseen data.
   - Increasing the number of iterations can exacerbate this issue, as the model will continue to learn the specifics of the limited data, worsening overfitting.

### 2. **Lack of Diversity**:
   - A small dataset might not represent the diversity of the data distribution well. Even with more iterations, the model may not learn generalized patterns because the training data is too limited in scope.
   - More iterations on a small dataset won't introduce new information; the model will just repeatedly see the same limited examples, which doesn't improve its ability to generalize.

### 3. **Validation Set Issues**:
   - A small validation set might not provide a reliable estimate of the model's performance. The model's accuracy could vary significantly with slight changes in the validation data due to the small sample size.
   - This makes it hard to gauge whether additional iterations are actually improving the model's performance or just overfitting to the training data.

### 4. **Possible Solutions**:
   - **Data Augmentation**: For image or text data, techniques like data augmentation can help by artificially increasing the size of the training dataset through transformations (e.g., rotations, scaling, or synonym replacement). This can help the model generalize better.
   - **Transfer Learning**: Using a pre-trained model and fine-tuning it on the small dataset can be effective. The pre-trained model already has learned general features, so it needs less data to adapt to the specific task.
   - **Regularization**: Techniques like dropout, L2 regularization, or early stopping can help reduce the risk of overfitting, even with more iterations.
   - **Cross-Validation**: Using k-fold cross-validation allows better use of a small dataset by splitting it into multiple training/validation sets, providing a more robust estimate of model performance.

In summary, while increasing iterations might seem like a straightforward solution, it can lead to overfitting and won't address the core issues of having too little data. Applying techniques like data augmentation, transfer learning, or regularization are more effective strategies in such scenarios.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.
Answer:-The exercise you’re referring to seems to involve evaluating the accuracy of k-Nearest Neighbors (k-NN) classifiers, particularly comparing the 3-nearest neighbors (3-NN) classifier with the 1-nearest neighbor (1-NN) classifier. Here's a breakdown of how you can approach this:

### 1. **Understanding the Key Concepts:**
   - **k-Nearest Neighbors (k-NN):** A non-parametric method used for classification and regression. The output is a class membership for classification. The class with the majority of votes from the nearest k neighbors is selected.
   - **Number of Splits:** This could refer to cross-validation splits (e.g., k-fold cross-validation) or the data splitting strategy.
   - **Split Size:** The size of the training and test sets, which impacts model performance and generalization.

### 2. **Experimental Setup:**
   - **Dataset:** Choose a dataset to work with (e.g., Iris dataset, digits, etc.).
   - **Splitting the Data:** Use different split sizes (e.g., 70-30, 80-20, or k-fold cross-validation) to evaluate how varying the training and test set sizes affect the accuracy.
   - **Classification Algorithms:**
     - **1-NN:** The model predicts the class of a new sample based on the closest single neighbor.
     - **3-NN:** The model predicts based on the majority vote among the three nearest neighbors.

### 3. **Steps to Implement:**
   1. **Load the Dataset:** Use a dataset from scikit-learn or load your own.
   2. **Implement the Classifiers:**
      - Use scikit-learn's `KNeighborsClassifier` with `n_neighbors=1` and `n_neighbors=3`.
   3. **Evaluate Accuracy with Different Splits:**
      - Perform different data splits (e.g., 70% training / 30% testing, 80% training / 20% testing).
      - Alternatively, use k-fold cross-validation to get a more robust estimate of accuracy.
   4. **Record the Results:**
      - Track the accuracy of both 1-NN and 3-NN classifiers for each split or fold.
   5. **Compare Results:** Analyze how the accuracy of 1-NN and 3-NN changes with different split sizes.

### 4. **Analysis:**
   - **Effect of Number of Splits:**
     - As the number of splits in cross-validation increases, the model might get more robust estimates of accuracy.
     - Smaller training sets might lead to overfitting, where 1-NN could perform better due to its simplicity, but it might also suffer from high variance.
   - **Comparison of 1-NN vs. 3-NN:**
     - 1-NN is more sensitive to noise because it relies on a single nearest neighbor.
     - 3-NN is generally more stable as it takes the majority vote among the three closest neighbors, leading to potentially better generalization.

### 5. **Conclusions:**
   - Compare the results and draw conclusions based on the data. For example, you might observe that 3-NN generally has better accuracy due to its robustness against noise, especially as the training set size decreases.

### Example Code Snippet (Python with scikit-learn):
```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Define classifiers
knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_3 = KNeighborsClassifier(n_neighbors=3)

# Split the data and evaluate
split_sizes = [0.7, 0.8, 0.9]
results = {}

for split_size in split_sizes:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split_size, random_state=42)
    
    knn_1.fit(X_train, y_train)
    knn_3.fit(X_train, y_train)
    
    acc_1nn = knn_1.score(X_test, y_test)
    acc_3nn = knn_3.score(X_test, y_test)
    
    results[f'Train/Test Split: {split_size}'] = {
        '1-NN Accuracy': acc_1nn,
        '3-NN Accuracy': acc_3nn
    }

# Evaluate using cross-validation
cv_scores_1nn = cross_val_score(knn_1, X, y, cv=5)
cv_scores_3nn = cross_val_score(knn_3, X, y, cv=5)

results['Cross-validation (5-fold)'] = {
    '1-NN CV Accuracy': np.mean(cv_scores_1nn),
    '3-NN CV Accuracy': np.mean(cv_scores_3nn)
}

print(results)
```

This code will give you a comparison between 1-NN and 3-NN with different training/testing splits and cross-validation.

### Expected Outcome:
- You might observe that the 3-NN classifier generally performs better than the 1-NN, especially when the training set size is small, as it is less prone to noise.
- As the number of splits in cross-validation increases, you might notice more consistent accuracy results for both classifiers.

This approach should help you understand how the accuracy of these classifiers changes with different splits and how they compare to each other.